In [1]:
from data_processor import TKGProcessor
import sys
import os
import pickle
from Encoders import *
from transformers import ElectraTokenizer, ElectraModel, DebertaV2Tokenizer, DebertaV2Model

d:\SWSN\swsn_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
data='data/inductive/all-triples/YAGO11k'#'data/inductive/all-triples/WIKIDATA12k' - inductive prediction, for transductive prediction use data/WIKIDATA12k 

The data is stored in the test.txt, train.txt and valid.txt files. Each file is composed of 5 columns: subjectID, predicateID, objectID, time1, time2 (a triple and a time interval). The corresponding entities and names are stored in additional files (entity2id.txt, entity2name.txt, entity2description.txt, relation2id.txt, relation2name.txt). Entity2id.txt contains an entity, id, time1, time2. 

8276	9	8277	-405-##-##	####-##-## //from test.txt 


\<Aristophanes\>	8276	-446-##-##	-386-##-## //from entity2id.txt

\<created\>	9 //from relation2itd.txt

\<The_Frogs\>	8277	####-##-##	####-##-## //from entity2id.txt

Now we have a triple \<Aristophanes\> \<created\> \<The_Frogs\> and a time interval -405-##-##	####-##-##.

\<Aristophanes\>	Aristophanes //from entity2name.txt
- \<Aristophanes\>	Aristophanes (/ˌærɪˈstɒfəniːz/; Ancient Greek: Ἀριστοφάνης, pronounced [aristopʰánɛːs]; c. 446 – c. 386 BC), son of Philippus, of the deme Kydathenaion (Latin: Cydathenaeum), was a comic playwright or comedy-writer of ancient Athens and a poet of Old Attic Comedy. Eleven of his forty plays survive virtually complete. These provide the most valuable examples of a genre of comic drama known as Old Comedy and are used to define it, along with fragments from dozens of lost plays by Aristophanes and his contemporaries.Also known as "The Father of Comedy" and "the Prince of Ancient Comedy", Aristophanes has been said to recreate the life of ancient Athens more convincingly than any other author. His powers of ridicule were feared and acknowledged by influential contemporaries; Plato singled out Aristophanes' play The Clouds as slander that contributed to the trial and subsequent condemning to death of Socrates, although other satirical playwrights had also caricatured the philosopher. //from entity2desc.txt

\<The_Frogs\>	The Frogs //from entity2name.txt
- \<The_Frogs\>	The Frogs (Greek: Βάτραχοι, translit. Bátrakhoi, lit. "Frogs"; Latin: Ranae, often abbreviated Ran. or Ra.) is a comedy written by the Ancient Greek playwright Aristophanes. It was performed at the Lenaia, one of the Festivals of Dionysus in Athens, in 405 BC and received first place.The Frogs tells the story of the god Dionysus, who, despairing of the state of Athens' tragedians, travels to Hades (the underworld) to bring the playwright Euripides back from the dead. (Euripides had died the year before, in 406 BC.) He brings along his slave Xanthias, who is smarter and braver than Dionysus. As the play opens, Xanthias and Dionysus argue over what kind of jokes Xanthias can use to open the play. For the first half of the play, Dionysus routinely makes critical errors, forcing Xanthias to improvise in order to protect his master and prevent Dionysus from looking incompetent—but this only allows Dionysus to continue to make mistakes with no consequence.To find a reliable path to Hades, Dionysus seeks advice from his half-brother Heracles, who had been there before in order to retrieve the hell hound Cerberus. Dionysus shows up at his doorstep dressed in a lion-hide and carrying a club. Heracles, upon seeing the effeminate Dionysus dressed up like himself, can't help laughing. When Dionysus asks which road is the quickest to get to Hades, Heracles tells him that he can hang himself, drink poison or jump off a tower. Dionysus opts for the longer journey, which Heracles himself had taken, across a lake (possibly Lake Acheron).  //from entity2desc.txt




In [3]:
processor=TKGProcessor(
    data_dir=data,
    task='lp', #available options: lp - link prediction, tp-time prediction
    mode='train', #train or test modes
    min_time=-453,
    max_time=2844, # min and max time taken from TEMT github
    sentence_encoder_model = TextEncoder(SentenceTransformerEncoder("sentence-transformers/all-mpnet-base-v2"), target_dim=None),
)

In [4]:
if os.path.exists('saved_features_yago_ind.dat'):

    with open("saved_features_yago_ind.dat", "rb") as f:
        train_features = pickle.load(f)
else:
    train_examples = processor.get_train_examples(data)
    train_features = processor.convert_examples_to_features(train_examples, use_descriptions=True)

    with open("saved_features_yago_ind.dat", "wb") as f:
        pickle.dump(train_features, f)

The processor produces train examples in the form of QuadrupleExample which is described like this:

Args:
-            guid: Unique id for the example.
-           subject_id: string.
-            predicate_id: string.
-            object_id: string.
-            timestamp: string
-            label: int. The label of the example. This should be
-            specified for train and dev examples, but not for test examples.

These examples are then transformed into features by encoding the time using a timeencoder and encoding the triple using an lm encoder.

In [5]:
train_examples = processor.get_train_examples(data)
train_examples[0]

In [6]:
train_features[0].time_encoding

array([-0.99386534,  0.11059696,  0.21958619,  0.97559311,  0.99969373,
        0.02474776,  0.52637916, -0.85024995,  0.25935578,  0.96578185,
       -0.99324366,  0.11604752,  0.40739265,  0.9132531 ,  0.26620496,
        0.96391645,  0.98589406, -0.16737056, -0.95766423, -0.28788752,
       -0.15398943, -0.98807249,  0.77520791,  0.63170617,  0.60880552,
        0.79331951,  0.46432282,  0.88566603, -0.93725695,  0.34863936,
        0.60936851,  0.79288714, -0.88324651,  0.46890895, -0.72745132,
        0.68615929,  0.81800738,  0.57520773, -0.65714359, -0.75376542,
        0.96923228,  0.24614791, -0.54805032,  0.83644536, -0.90660564,
       -0.42197893, -0.06547951, -0.99785391,  0.67176778, -0.7407618 ,
        0.97304748, -0.23060486,  0.97625285,  0.21663417,  0.84904767,
        0.52831625,  0.6893039 ,  0.72447231,  0.53989754,  0.84173075,
        0.41475742,  0.90993202,  0.31524225,  0.94901124])

In [7]:
train_features[0].triple_encoding

array([ 1.00260656e-02,  6.02931790e-02,  2.31985971e-02,  5.43634780e-02,
       -4.37507480e-02, -3.63928378e-02,  2.70555820e-02, -2.82605384e-02,
       -7.47601464e-02, -1.87179074e-02, -4.87498287e-03,  1.94866247e-02,
       -1.09955063e-02, -3.89467739e-03,  7.56431669e-02,  5.42912595e-02,
       -5.71947685e-03, -4.27759290e-02, -3.48124504e-02,  3.68406512e-02,
        9.78542143e-04,  9.77070928e-02, -8.84434730e-02,  4.92405705e-03,
       -4.69758771e-02, -3.17334495e-02,  1.70206348e-03, -2.09851637e-02,
        4.90555204e-02,  3.63717526e-02,  2.78491825e-02, -2.72866851e-03,
        3.52835050e-03,  1.56465098e-02,  2.16209014e-06, -7.72048626e-03,
       -1.95517130e-02,  4.22785394e-02, -4.40586060e-02,  1.31802196e-02,
        7.23850653e-02,  4.53405194e-02, -8.09378847e-02,  2.66321227e-02,
       -2.35456601e-02, -2.89671998e-02, -1.42388009e-02, -3.21502797e-02,
       -1.20003987e-02,  5.11814244e-02, -1.49555178e-02, -1.56336632e-02,
       -9.19021368e-02, -